In [1]:
import os
from os.path import join as pjoin

import numpy as np
import pandas as pd
from datetime import datetime
from glob import glob
from Utilities.dynarray import DynamicRecArray as recarray
from Utilities.metutils import convert
# IDY03101 - METAR data

In [2]:
def parseTime(datestr, timestr):
    return datetime.strptime("{0} {1}".format(datestr, timestr), "%Y%m%d %H%M")

METAR_DTYPE=[("stnWMO", '|S5'), ("stnCode", "|S4"), ("dtDate", "|S8"), ("dtTime", "|S4"), 
       ("stnLat", "f8"), ("stnLon", "f8"), ("winddir", "f8"), ("windspeed", "f8"), 
       ("tempDB", "f8"), ("dewpt", "f8"), ("QNH", "f8"), ("RF9am", "f8"), ("RF10min", "f8"),
       ("vis", "f8"), ("Avis", "f8"), ("gust", "f8")]
METAR_NAMES = [field[0] for field in METAR_DTYPE]
METAR_COLS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
METAR_CONV = {'stnCode': lambda s: s.strip('"'),
              'windspeed': lambda s: convert(s, "kts", "mps"),
              'gust': lambda s: convert(s, "kts", "mps")}

SYNOP_DTYPE = [('stnWMO', '|S5'), ('stnName', '|S31'), ('stnCode', '|S4'), ('dtDate', '|S8'), 
               ('dtTime', '|S4'), ('stnLat', 'f8'), ('stnLon', 'f8'), ('winddir', 'f8'), 
               ('windspeed', 'f8'), ('vis', 'i'), ('presentWxCode', '|S8'), ('pastWxCode', 'i'),
               ('mslp', 'f8'), ('tempDB', 'f8'), ('dewpt', 'f8')]
SYNOP_NAMES = [field[0] for field in SYNOP_DTYPE]
SYNOP_COLS = [0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 16, 21, 22]
SYNOP_CONV = {'stnName': lambda s: s.strip('"').rstrip(' '),
              'stnCode': lambda s: s.strip('"').rstrip(' '),
              'presentWxCode': lambda s: s.strip('"').rstrip(' ')}

inputPath = "N:/climate_change/CHARS/B_Wind/data/raw/obs/axf/"
outputPath = "N:/climate_change/CHARS/B_Wind/data/derived/obs/"

In [3]:
metarbasename = "IDY03101.*.axf"

metardata = {}

filelist = glob(pjoin(inputPath, metarbasename))
for f in filelist:
    arr = np.genfromtxt(f, dtype=METAR_DTYPE, delimiter=',', skip_header=2,
                        skip_footer=1, usecols=METAR_COLS, names=METAR_NAMES,
                        autostrip=True, converters=METAR_CONV)
    for i in range(len(arr)):
        key = "{0}-{1}".format(arr["stnWMO"][i], arr['stnCode'][i])
        if metardata.has_key(key):
            metardata[key] = np.append(metardata[key], arr[i])
        else:
            metardata[key] = recarray(METAR_DTYPE)
            metardata[key] = arr[i]

In [37]:
metarfmt = ['%s', '%s', '%s', '%s', '%6.2f', '%6.2f', '%5.1f', 
           '%5.1f', '%5.1f', '%5.1f', '%6.1f', '%5.1f', '%5.1f', 
           '%7.1f', '%7.1f', '%6.1f']
for key in metardata.keys():
    fname = pjoin(outputPath, "metar/{0}.csv".format(key))
    df = pd.DataFrame(metardata[key])
    dfdup = df.drop_duplicates(['dtDate', 'dtTime'])
    da = dfdup.to_records(index=False)
    np.savetxt(fname, da, fmt=metarfmt, delimiter=',')

In [18]:
synopbasename = "IDY03000.*.axf"
synopdata = {}

filelist = glob(pjoin(inputPath, synopbasename))
for f in filelist:
    try:
        arr = np.genfromtxt(f, dtype=SYNOP_DTYPE, delimiter=',', skip_header=2,
                            skip_footer=1, usecols=SYNOP_COLS, names=SYNOP_NAMES,
                            autostrip=True, converters=SYNOP_CONV)
    except IndexError:
        print f
    if arr.size == 1:
        key = "{0}-{1}".format(arr["stnWMO"], arr['stnCode'])
        if synopdata.has_key(key):
            synopdata[key] = np.append(synopdata[key], arr)
        else:
            synopdata[key] = recarray(SYNOP_DTYPE)
            synopdata[key] = arr
    else:
        for i in range(len(arr)):
            key = "{0}-{1}".format(arr["stnWMO"][i], arr['stnCode'][i])
            if synopdata.has_key(key):
                synopdata[key] = np.append(synopdata[key], arr[i])
            else:
                synopdata[key] = recarray(SYNOP_DTYPE)
                synopdata[key] = arr[i]

In [35]:
synopfmt = ['%s', '%s', '%s', '%s', '%s', '%6.2f', '%6.2f', '%5.1f', 
            '%5.1f', '%d', '%s', '%d', '%6.1f', '%4.1f', '%4.1f']
for key in synopdata.keys():
    fname = pjoin(outputPath, "synop/{0}.csv".format(key))
    try:
        df = pd.DataFrame(synopdata[key])
        dfdup = df.drop_duplicates(['dtDate', 'dtTime'])
        da = dfdup.to_records(index=False)
        np.savetxt(fname, da, fmt=synopfmt, delimiter=',')
    except IndexError:
        print key